<a href="https://colab.research.google.com/github/andivaanggoro/andivaanggoromachinelearning/blob/main/UasMachineLearning_Andiva_Kasih_Anggoro_Putra_1103204031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Andiva Kasih Anggoro Putra

NIM : 1103204031

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

# The below 3 Installation was required in Kaggle Kernel
# !pip install --upgrade --force-reinstall --no-deps torch
# !pip install --upgrade --force-reinstall --no-deps torchvision
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
from tqdm import tqdm
import time
import copy
import timm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import timm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device='cpu'
device

device(type='cpu')

In [4]:
device(type='cuda')

TypeError: 'torch.device' object is not callable

In [5]:
print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [6]:
# Colab path from G-Drive
ROOT_PATH = "/content/drive/MyDrive/Statistik/SplitOver"

# Kaggle Root Path
# ROOT_PATH = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/"

# Local Machine
# ROOT_PATH = "../../Datasets/Covid-19-Image-Dataset-170MB/"

#  Image size is not same throughout the whole dataset. Therefore all the images has been resized to 256 X 256 size using the Resize() function

**Plotting Images**

mengakses direktori yang berisikan data train untuk berbagai jenis tanaman dan menampilkan nama file dari setiap jenis tanaman tersebut dengan mendefinisikan path direktori dengan ROOT_PATH (yang merupakan path root) dengan sub directori 'train'.

In [8]:
train_dir = os.path.join(ROOT_PATH, 'train')

train_kangkungair_dir = os.path.join(train_dir, 'Kangkung_Air')
train_kangkungdarat_dir = os.path.join(train_dir, 'Kangkung_Darat')
train_bayam_dir = os.path.join(train_dir, 'Bayam')
train_kelor_dir = os.path.join(train_dir, 'Kelor')
train_pakis_dir = os.path.join(train_dir, 'Pakis')

train_kangkungair_fnames = os.listdir(train_kangkungair_dir)
train_kangkungdarat_fnames = os.listdir(train_kangkungdarat_dir)
train_bayam_fnames = os.listdir(train_bayam_dir)
train_kelor_fnames = os.listdir(train_kelor_dir)
train_pakis_fnames = os.listdir(train_pakis_dir)

print(train_kangkungair_fnames[:10])
print(train_kangkungdarat_fnames[:10])
print(train_bayam_fnames[:10])
print(train_kelor_fnames[:10])
print(train_pakis_fnames[:10])

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Statistik/SplitOver/train/Kangkung_Air'

In [9]:
def get_dataset_loader(data_path, batch_size=256, train=False):
  """  Get the dataset loader for training or testing.

    Parameters:
    - data_path (str): Path to the dataset directory.
    - batch_size (int): Batch size for the dataloader.
    - train (bool): Whether to get the training dataset loader or testing dataset loader.

    Returns:
    - dataloader (torch.utils.data.DataLoader): Dataset loader.
    - dataset_size (int): Size of the dataset. """
  if train:
    transform = transforms.Compose([
      transforms.RandomHorizontalFlip(p=0.5),
      transforms.RandomVerticalFlip(p=0.5),
      transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(), transforms.GaussianBlur(3)]), p=0.1),
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize((0.485, 0.456, 0.466), (0.229, 0.224, 0.255)),
      transforms.RandomErasing(p=0.12, value='random')
    ])

    train_imgs = datasets.ImageFolder(os.path.join(ROOT_PATH, 'train/'), transform=transform)

    print(f"We have {len(train_imgs)} Training Images with {len(train_imgs.classes)} classes")

    len_train_images = int(len(train_imgs) * 0.78)

    len_validation_images = int(len(train_imgs) - len_train_images )

    train_dataset, validation_dataset = random_split(train_imgs, [len_train_images, len_validation_images] )

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    return train_dataloader, validation_dataloader, len(train_dataset), len(validation_dataset)

  else:
    transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.466), (0.229, 0.224, 0.255)),
  ])

  test_imgs = datasets.ImageFolder(os.path.join(ROOT_PATH, 'test/'), transform=transform)

  print(f"We have {len(test_imgs)} Training Images with {len(test_imgs.classes)} classes")

  test_dataloader = DataLoader(test_imgs, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

  return test_dataloader, len(test_imgs)

**Get Number of Classes**

In [10]:
class_categories = datasets.ImageFolder(os.path.join(ROOT_PATH, 'train/')).classes
# ['Covid', 'Normal', 'Viral Pneumonia']
num_classes = len(class_categories)
num_classes

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Statistik/SplitOver/train/'

**Display a random image**

In [11]:
train_data = datasets.ImageFolder(os.path.join(ROOT_PATH, 'train/'))
train_data[2][0]

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Statistik/SplitOver/train/'

In [12]:
BATCH_SIZE = 32

(train_dataloader, validation_dataloader, len_train_dataloader, len_validation_dataloader) = get_dataset_loader(ROOT_PATH, batch_size=BATCH_SIZE, train=True)

(test_dataloader, len_test_dataloader) = get_dataset_loader(ROOT_PATH, batch_size=BATCH_SIZE, train=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Statistik/SplitOver/train/'

In [13]:
train_dataloader.dataset

NameError: name 'train_dataloader' is not defined

In [14]:
test_dataloader.dataset

NameError: name 'test_dataloader' is not defined

In [15]:
print(len(train_dataloader))
print(len(validation_dataloader))
print(len(test_dataloader))
print(len_train_dataloader, len_validation_dataloader, len_test_dataloader )


NameError: name 'train_dataloader' is not defined

In [17]:
dataloaders_dict = {
  'train': train_dataloader,
  'validation': validation_dataloader
}

dataset_sizes_dict = {
  'train':len_train_dataloader,
  'validation': len_validation_dataloader
}

NameError: name 'train_dataloader' is not defined

In [18]:
torch.backends.cudnn.benchmark = True

model = models.efficientnet_b0(pretrained=True)

print("model.classifier is => ", model.classifier)

""" model.classifier is =>  Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)
"""

for param in model.parameters():
  param.requires_grad = False

num_inputs = model.classifier[1].in_features


""" Now here is where I am adding additional layers to the pre-trained model.

for nn.Linear - note the following
in_features – size of each input sample
out_features – size of each output sample

"""


model.classifier = nn.Sequential(
  nn.Linear(num_inputs, 2048),
  nn.SiLU(), # Sigmoid Weighted Linear Unit
  nn.Dropout(0.2),
  # Note that the last layer is 2048 * Number of Classes
  # Reshape the final layer(s) to have the same number of outputs as the number of classes in the new dataset
  nn.Linear(2048, len(class_categories))
)

model = model.to(device)
print(model.classifier)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 132MB/s]


model.classifier is =>  Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)


NameError: name 'class_categories' is not defined

In [19]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.11)
criterion = criterion.to(device)
optimizer = optim.AdamW(model.classifier.parameters(), lr=0.001 )

In [20]:
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
# Change the learning rate based on number of epochs
# Decays the learning rate of each parameter group by gamma every step_size epochs


In [21]:
training_history = {'accuracy': [], 'loss':[] }
validation_history = {'accuracy': [], 'loss': []}

In [22]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
  start_time = time.time()

  best_model_weights = copy.deepcopy(model.state_dict())
  best_accuracy = 0.0

  for epoch in range(num_epochs):
    print('Running epoch {}/{}'.format(epoch, num_epochs-1))
    print('-' * 10)

    """ Within each epoch first configure training or eval mode
    based on wheather I am in the 'train' or 'validation' phase.

    `model.train()` tells your model that you are training the model. So effectively layers like dropout, batchnorm etc.
     which behave different on the train and test procedures know what is going on and hence can behave accordingly.

    In below, after I start the for loop for phase, the rest entire code for this train() function runs
    within this for loop and at various stage passing the phase variable to be either 'train' or
    'validation'

    """
    for phase in ['train', 'validation']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0

      """ now start the iteration of the dataloaders_dict variable
      i.e. for both 'train' and 'validation' phases

      Below loop, basically tells that, for every epoch the train_dataloader and then the validation_dataloader is invoked
       which returns input and its corresponding label.

      """
      for inputs, labels in tqdm(dataloaders_dict[phase]):
        inputs = inputs.to(device)
        labels = labels.to(device)

        """ for every mini-batch during the training phase, we typically want to explicitly set the gradients
        to zero before starting to do backpropragation """
        optimizer.zero_grad()

        """ Start the forward movement
        tracking history for phase == 'train'
        set_grad_enabled is a Context-manager that sets gradient calculation to on or off.

        The argument to it is called mode (which is a boolean ), and this mode controls whether to enable grad (True),
        or disable (False). This can be used to conditionally enable gradients.
        """
        with torch.set_grad_enabled(phase == 'train'):
          predicted_outputs = model(inputs)
          # print('outputs on which to apply torch.max ', outputs)
          # find the maximum along the rows, use dim=1 to torch.max()
          _, predictions = torch.max(predicted_outputs, 1)

          loss = criterion(predicted_outputs, labels)

          # Now for the 'train' phase, do backpropagation and update weights with step()
          if phase == 'train':
            loss.backward()
            optimizer.step()

        """ Update the running loss and corrects
        we need to use, loss.item() instead of loss alone in running_loss calculation and averaging.
        Because loss gives you a grad_function, not a float value.
        The item() method extracts the loss’s value as a Python float.
        """
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(predictions == labels.data)

      if phase == 'train':
        scheduler.step()

      epoch_loss = running_loss / dataset_sizes_dict[phase]
      epoch_accuracy = running_corrects.double() / dataset_sizes_dict[phase]

      """ To summarize, in the above code what I did is

      1. => multiply each average batch loss with batch-length.
      The batch-length is inputs.size(0) which gives the number total images in each batch.
      Essentially I am un-averaging the Batch-Loss

      Do this all the batches inside the batch-running loop.
      And then after that loop, i.e. outside the batch-loop and coming back to my epoch-loop

      2. => Divide this accumulated un-averaged Batch-loss from all batches,
      by the number of samples (len(train_dataloader)) to get the exact train loss average for the epoch """


      if phase == 'train':
        training_history['loss'].append(epoch_loss)
        training_history['accuracy'].append(epoch_accuracy)
      elif phase == 'validation':
        validation_history['loss'].append(epoch_loss)
        validation_history['accuracy'].append(epoch_accuracy)

      # finally update best_accuracy and deep copy the model
      if phase == 'validation' and epoch_accuracy > best_accuracy:
        best_accuracy = epoch_accuracy
        best_model_weights = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - start_time
  print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60
  ))
  print('Best Validation Accuracy: {:4f}'.format(best_accuracy))

  # loading best model weights
  model.load_state_dict(best_model_weights)
  return model

In [23]:
model_trained = train(model, criterion, optimizer, exp_lr_scheduler, num_epochs=150)

Running epoch 0/149
----------


NameError: name 'dataloaders_dict' is not defined

In [24]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from tqdm import tqdm

def evaluate(model, test_dataloader, criterion, class_categories, BATCH_SIZE):
    loss_on_test_dataset = 0.0
    correct_class = list(0. for i in range(len(class_categories)))
    total_correct_for_all_classes = list(0. for i in range(len(class_categories)))

    all_preds = []
    all_labels = []

    model.eval()

    for test_inputs, test_labels in tqdm(test_dataloader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()

        with torch.no_grad():
            output_test = model(test_inputs)
            loss = criterion(output_test, test_labels)

        loss_on_test_dataset += loss.item() * test_inputs.size(0)
        _, pred_test = torch.max(output_test, 1)

        all_preds.extend(pred_test.cpu().numpy())
        all_labels.extend(test_labels.cpu().numpy())

        correct_tensor = pred_test.eq(test_labels.view_as(pred_test))
        correct = np.squeeze(correct_tensor.cpu().numpy()) if torch.cuda.is_available() else np.squeeze(correct_tensor.numpy())

        if len(test_labels) == BATCH_SIZE:
            for i in range(BATCH_SIZE):
                label = test_labels.data[i]
                correct_class[label] += correct[i].item()
                total_correct_for_all_classes[label] += 1

    loss_on_test_dataset = loss_on_test_dataset/len(test_dataloader.dataset)
    print('Test Loss: {:.6f}\n'.format(loss_on_test_dataset))

    for i in range(len(class_categories)):
        if total_correct_for_all_classes[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                class_categories[i], 100 * correct_class[i] / total_correct_for_all_classes[i],
                np.sum(correct_class[i]), np.sum(total_correct_for_all_classes[i])
            ))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (class_categories[i]))

    print('\nTest Accuracy (Overall): {:.4f} ({}/{})'.format(
        100 * np.sum(correct_class) / np.sum(total_correct_for_all_classes),
        np.sum(correct_class), np.sum(total_correct_for_all_classes)
    ))

    # Calculate additional metrics
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1 Score: {:.4f}'.format(f1))

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_categories, yticklabels=class_categories)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Assuming model_trained, test_dataloader, criterion, class_categories, and BATCH_SIZE are defined
evaluate(model_trained, test_dataloader, criterion, class_categories, BATCH_SIZE)

NameError: name 'model_trained' is not defined